In [1]:
import glob
import oyaml as yaml
import numpy as np
import pandas as pd

In [2]:
import zinv
help(zinv.modules.analyse)

Welcome to JupyROOT 6.18/00
Help on function analyse in module zinv.modules.analyse:

analyse(dataset_cfg, sequence_cfg, event_selection_cfg, physics_object_cfg, trigger_cfg, hdf_cfg, name='zinv', outdir='output', tempdir='_ccsp_temp', mode='multiprocessing', batch_opts='-q hep.q', ncores=0, nblocks_per_dataset=-1, nblocks_per_process=-1, nfiles_per_dataset=-1, nfiles_per_process=1, blocksize=1000000, cachesize=8, quiet=False, sample=None)



In [3]:
help(zinv.modules.resume)

Help on function resume in module zinv.modules.resume:

resume(path, batch_opts='-q hep.q', sleep=5, request_resubmission_options=True)



In [4]:
#zinv.modules.analyse(
#    "configs/nominal/datasets.yaml",
#    "configs/nominal/module_sequence.yaml",
#    "configs/nominal/event_selection.yaml",
#    "configs/nominal/object_selection.yaml",
#    "configs/nominal/trigger_selection.yaml",
#    "configs/nominal/hdf_output.yaml",
#    outdir="/vols/cms/sdb15/ZinvWidth/tables/results_v2/",
#    tempdir="/vols/cms/sdb15/ZinvWidth/tables/_ccsp_temp/",
#    mode="sge",
#    batch_opts="-q hep.q -l h_rt=3:0:0 -l h_vmem=12G",
#    #mode="multiprocessing",
#    #ncores=0,
#    nblocks_per_process=4,
#    blocksize=1_000_000,
#    sample="data",
#)

#zinv.modules.resume(
#    "/vols/cms/sdb15/ZinvWidth/tables/_ccsp_temp/tpd_20190813_231556_deu_fsr3",
#    batch_opts="-q hep.q -l h_rt=3:0:0 -l h_vmem=24G",
#)

## MET Skims

In [5]:
import zdb
help(zdb.modules.skim)

Help on function skim in module zdb.modules.skim:

skim(config, mode='multiprocessing', ncores=0, nfiles=-1, batch_opts='', output=None)



In [6]:
!cat configs/met_skim/cfg_data.yaml | head -n 10

selection: "(IsCertified) & (IsMETTriggered | IsSingleMuonTriggered | IsSingleElectronTriggered) & (Flag_goodVertices) & (Flag_globalSuperTightHalo2016Filter) & (Flag_HBHENoiseFilter) & (Flag_HBHENoiseIsoFilter) & (Flag_EcalDeadCellTriggerPrimitiveFilter) & (Flag_BadPFMuonFilter) & (Flag_BadGlobalMuon) & (Flag_CloneGlobalMuon) & (Flag_eeBadScFilter) & (MET_dCaloMET<0.6) & (nJetSelection>0) & (nJetSelection==nJetVeto) & (LeadJetSelection_pt>200.) & (LeadJetSelection_chHEF>0.1) & (LeadJetSelection_chHEF<0.95) & (nPhotonVeto==0) & (nBJetVeto==0)"
tables:
    "central": "Events"
files:
    - "/vols/cms/sdb15/Analysis/ZinvWidth/databases/201906_Jun/14_sortpt/Data/result_00000.h5"
    - "/vols/cms/sdb15/Analysis/ZinvWidth/databases/201906_Jun/14_sortpt/Data/result_00001.h5"
    - "/vols/cms/sdb15/Analysis/ZinvWidth/databases/201906_Jun/14_sortpt/Data/result_00002.h5"
    - "/vols/cms/sdb15/Analysis/ZinvWidth/databases/201906_Jun/14_sortpt/Data/result_00003.h5"
    - "/vols/cms/sdb15/Analysis

In [7]:
met_triggers = [
    "HLT_PFMETNoMu90_PFMHTNoMu90_IDTight",
    "HLT_PFMETNoMu100_PFMHTNoMu100_IDTight",
    "HLT_PFMETNoMu110_PFMHTNoMu110_IDTight",
    "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight",
    "(HLT_PFMET170_NotCleaned & (run<276282))",
    "(HLT_PFMET170_BeamHaloCleaned & (run<276282))",
    "HLT_PFMET170_HBHECleaned",
    "HLT_PFMET170_HBHE_BeamHaloCleaned",
]

muon_triggers = [
    "HLT_IsoMu24",
    "HLT_IsoTkMu24",
]

selection = [
    "IsCertified",
    "(" + " | ".join(muon_triggers) + ")",
    "Flag_goodVertices",
    "Flag_globalSuperTightHalo2016Filter",
    "Flag_HBHENoiseFilter",
    "Flag_HBHENoiseIsoFilter",
    "Flag_EcalDeadCellTriggerPrimitiveFilter",
    "Flag_BadPFMuonFilter",
    "MET_dCaloMET<0.6",
    "nJetSelection>0",
    "nJetSelection==nJetVeto",
    "LeadJetSelection_pt>200.",
    "LeadJetSelection_chHEF>0.1",
    "LeadJetSelection_neHEF<0.8",
    "nPhotonVeto==0",
    "nBJetVeto==0",
] 

cfg = {
    "selection": "(" + ") & (".join(selection) + ")",
    "tables": {"central": "Events"},
    "files": sorted(p for p in glob.glob("/vols/cms/sdb15/ZinvWidth/tables/results_v2/*.h5")),
}
with open("configs/met_skim/data.yaml", "w") as f:
    yaml.dump(cfg, f, indent=4)

In [8]:
!cat configs/met_skim/data.yaml | head -n 10

selection: (IsCertified) & ((HLT_IsoMu24 | HLT_IsoTkMu24)) & (Flag_goodVertices) &
    (Flag_globalSuperTightHalo2016Filter) & (Flag_HBHENoiseFilter) & (Flag_HBHENoiseIsoFilter)
    & (Flag_EcalDeadCellTriggerPrimitiveFilter) & (Flag_BadPFMuonFilter) & (MET_dCaloMET<0.6)
    & (nJetSelection>0) & (nJetSelection==nJetVeto) & (LeadJetSelection_pt>200.) &
    (LeadJetSelection_chHEF>0.1) & (LeadJetSelection_neHEF<0.8) & (nPhotonVeto==0)
    & (nBJetVeto==0)
tables:
    central: Events
files:
- /vols/cms/sdb15/ZinvWidth/tables/results_v2/result_00000.h5
cat: write error: Broken pipe


In [9]:
help(zdb.modules.skim)

Help on function skim in module zdb.modules.skim:

skim(config, mode='multiprocessing', ncores=0, nfiles=-1, batch_opts='', output=None)



In [10]:
!mkdir -p results/met_skims

In [11]:
zdb.modules.skim(
    "configs/met_skim/data.yaml",
    #mode="multiprocessing",
    #ncores=0,
    mode='sge',
    ncores=400,
    batch_opts='-q hep.q',
    output="/vols/build/cms/sdb15/ZinvWidth/zinv-notebooks/notebooks/corrections/triggers/results/met_skims/result_{:05d}.h5",
)

2019-08-14 10:32:35,557 - pysge.area - INFO - Creating paths in /vols/build/cms/sdb15/ZinvWidth/zinv-notebooks/notebooks/corrections/triggers/_ccsp_temp/tpd_20190814_103235_n4etndo7


2019-08-14 10:32:39,688 - pysge.submitter - ERROR - Malformed qsub submission string: ''


AttributeError: 'NoneType' object has no attribute 'group'